# Modélisation - Challenge Titanic

Ce notebook vous guide à travers le processus de modélisation pour prédire la survie des passagers du Titanic. Nous allons tester différents algorithmes de classification et optimiser leurs performances.

## Objectifs de la modélisation:
- Tester plusieurs algorithmes de classification
- Évaluer les performances des modèles avec validation croisée
- Optimiser les hyperparamètres des modèles les plus prometteurs
- Sélectionner le meilleur modèle pour les prédictions finales

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pour la modélisation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

# Algorithmes de classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Pour charger les données prétraitées
from sklearn.externals import joblib
import os

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
sns.set(font_scale=1.2)

# Pour afficher les graphiques dans le notebook
%matplotlib inline

# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

## 1. Chargement des données prétraitées

Commençons par charger les données prétraitées lors de l'étape précédente.

In [ ]:
# Chemin vers les données prétraitées
preprocessed_dir = '/workspaces/titanicML/Data/preprocessed'
preprocessed_file = os.path.join(preprocessed_dir, 'preprocessed_data.pkl')

try:
    # Charger les données prétraitées
    preprocessed_data = joblib.load(preprocessed_file)
    
    # Extraire les éléments du dictionnaire
    X_train = preprocessed_data['train_features']
    y_train = preprocessed_data['train_target']
    X_test = preprocessed_data['test_features']
    test_passenger_ids = preprocessed_data['test_passenger_ids']
    feature_names = preprocessed_data['feature_names']
    scaler = preprocessed_data['scaler']
    
    print("Données prétraitées chargées avec succès!")
    print(f"Ensemble d'entraînement: {X_train.shape[0]} observations, {X_train.shape[1]} caractéristiques")
    print(f"Ensemble de test: {X_test.shape[0]} observations, {X_test.shape[1]} caractéristiques")
    
except FileNotFoundError:
    print(f"Erreur: Fichier {preprocessed_file} introuvable.")
    print("Veuillez d'abord exécuter le notebook de prétraitement des données.")
    
    # Alternative: charger les données depuis les fichiers CSV
    try:
        train_file = os.path.join(preprocessed_dir, 'train_processed.csv')
        test_file = os.path.join(preprocessed_dir, 'test_processed.csv')
        
        train_data = pd.read_csv(train_file)
        X_test = pd.read_csv(test_file)
        
        # Séparer les caractéristiques et la cible dans les données d'entraînement
        y_train = train_data['Survived']
        X_train = train_data.drop('Survived', axis=1)
        
        # Récupérer les IDs des passagers de test depuis le fichier d'origine
        test_data_original = pd.read_csv('/workspaces/titanicML/Data/test.csv')
        test_passenger_ids = test_data_original['PassengerId']
        
        feature_names = X_train.columns.tolist()
        
        print("Données chargées depuis les fichiers CSV.")
        print(f"Ensemble d'entraînement: {X_train.shape[0]} observations, {X_train.shape[1]} caractéristiques")
        print(f"Ensemble de test: {X_test.shape[0]} observations, {X_test.shape[1]} caractéristiques")
        
    except FileNotFoundError:
        print("Erreur: Fichiers CSV également introuvables.")
        print("Veuillez exécuter le notebook de prétraitement des données avant de continuer.")

## 2. Création d'un ensemble de validation

Avant de commencer la modélisation, divisons notre ensemble d'entraînement pour créer un ensemble de validation qui nous permettra d'évaluer les performances de nos modèles.

In [ ]:
# Diviser les données en ensembles d'entraînement et de validation
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

print(f"Ensemble d'entraînement: {X_train_split.shape[0]} observations")
print(f"Ensemble de validation: {X_val.shape[0]} observations")

# Vérifier la distribution de la variable cible dans les ensembles d'entraînement et de validation
print("\nDistribution de la variable cible (Survived):")
print(f"Ensemble d'entraînement: {y_train_split.mean()*100:.2f}% de survivants")
print(f"Ensemble de validation: {y_val.mean()*100:.2f}% de survivants")

# Vérifier les dimensions des données
print("\nDimensions des ensembles de données:")
print(f"X_train_split: {X_train_split.shape}")
print(f"y_train_split: {y_train_split.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")

## 3. Évaluation de différents algorithmes de classification

Testons plusieurs algorithmes de classification pour identifier les plus prometteurs.

In [ ]:
# Définir une fonction pour évaluer un modèle avec validation croisée
def evaluate_model(model, X, y, cv=5):
    # Évaluer avec validation croisée
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    
    # Entraîner le modèle sur l'ensemble complet pour l'analyse des caractéristiques
    model.fit(X, y)
    
    # Retourner les résultats
    return {
        'model': model,
        'cv_scores': cv_scores,
        'mean_cv_score': cv_scores.mean(),
        'std_cv_score': cv_scores.std(),
        'trained_model': model  # Modèle entraîné sur l'ensemble complet
    }

# Définir les modèles à évaluer
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=500),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

# Évaluer chaque modèle avec validation croisée
model_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"Évaluation de {name}...")
    results = evaluate_model(model, X_train_split, y_train_split, cv=cv)
    model_results[name] = results
    print(f"  Précision moyenne (CV): {results['mean_cv_score']:.4f} ± {results['std_cv_score']:.4f}")

# Afficher les résultats sous forme de tableau
results_df = pd.DataFrame({
    'Modèle': list(model_results.keys()),
    'Précision moyenne (CV)': [results['mean_cv_score'] for results in model_results.values()],
    'Écart-type': [results['std_cv_score'] for results in model_results.values()]
})

results_df = results_df.sort_values('Précision moyenne (CV)', ascending=False).reset_index(drop=True)
display(results_df)

# Visualiser les résultats
plt.figure(figsize=(14, 7))
sns.barplot(x='Précision moyenne (CV)', y='Modèle', data=results_df, palette='viridis')
plt.title('Comparaison des performances des modèles (Validation croisée)', fontsize=16)
plt.xlabel('Précision moyenne', fontsize=14)
plt.ylabel('Modèle', fontsize=14)

# Ajouter les valeurs de précision sur les barres
for i, v in enumerate(results_df['Précision moyenne (CV)']):
    plt.text(v + 0.01, i, f"{v:.4f}", va='center', fontsize=12)

plt.xlim(0.75, 0.9)  # Ajuster les limites pour une meilleure visualisation
plt.tight_layout()
plt.show()

## 4. Analyse des caractéristiques importantes

Analysons l'importance des différentes caractéristiques pour comprendre ce qui influence le plus les prédictions.

In [ ]:
# Fonctions pour extraire l'importance des caractéristiques selon le type de modèle
def get_feature_importance(model, feature_names):
    if hasattr(model, 'feature_importances_'):  # Pour les modèles basés sur les arbres
        return pd.DataFrame({
            'Feature': feature_names,
            'Importance': model.feature_importances_
        }).sort_values('Importance', ascending=False)
    
    elif hasattr(model, 'coef_'):  # Pour les modèles linéaires
        # Utiliser la valeur absolue des coefficients comme mesure d'importance
        importances = np.abs(model.coef_[0])
        return pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
    
    else:
        return None

# Modèles pour lesquels nous pouvons analyser l'importance des caractéristiques
tree_based_models = ['Random Forest', 'Gradient Boosting', 'XGBoost', 'Decision Tree']
linear_models = ['Logistic Regression']

# Afficher l'importance des caractéristiques pour les modèles pertinents
for name in tree_based_models + linear_models:
    if name in model_results:
        model = model_results[name]['trained_model']
        importance_df = get_feature_importance(model, feature_names)
        
        if importance_df is not None:
            print(f"\nImportance des caractéristiques pour {name}:")
            display(importance_df.head(15))  # Afficher les 15 caractéristiques les plus importantes
            
            # Visualiser l'importance des caractéristiques
            plt.figure(figsize=(12, 8))
            sns.barplot(x='Importance', y='Feature', data=importance_df.head(15), palette='viridis')
            plt.title(f'Importance des caractéristiques pour {name}', fontsize=16)
            plt.xlabel('Importance', fontsize=14)
            plt.ylabel('Caractéristique', fontsize=14)
            plt.tight_layout()
            plt.show()

## 5. Optimisation des hyperparamètres des modèles les plus prometteurs

Optimisons les hyperparamètres des modèles les plus performants pour améliorer leurs performances.

In [ ]:
# Sélectionner les 3 modèles les plus performants pour l'optimisation
top_models = results_df['Modèle'].head(3).tolist()
print(f"Optimisation des hyperparamètres pour les modèles: {', '.join(top_models)}")

# Définir les grilles de paramètres pour l'optimisation
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 4, 5],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }
}

# Fonction pour optimiser les hyperparamètres avec RandomizedSearchCV
def optimize_hyperparameters(model_name, base_model, param_grid, X, y, cv=5, n_iter=20):
    print(f"\nOptimisation des hyperparamètres pour {model_name}...")
    
    # Utiliser RandomizedSearchCV pour l'efficacité
    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_grid,
        n_iter=n_iter,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    
    # Entraîner le modèle
    random_search.fit(X, y)
    
    # Afficher les meilleurs paramètres et score
    print(f"Meilleurs paramètres trouvés: {random_search.best_params_}")
    print(f"Meilleur score: {random_search.best_score_:.4f}")
    
    return random_search.best_estimator_, random_search.best_params_, random_search.best_score_

# Optimiser les modèles sélectionnés
optimized_models = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for model_name in top_models:
    if model_name in param_grids:
        base_model = models[model_name]
        param_grid = param_grids[model_name]
        
        best_model, best_params, best_score = optimize_hyperparameters(
            model_name, base_model, param_grid, X_train_split, y_train_split, cv=cv
        )
        
        optimized_models[model_name] = {
            'model': best_model,
            'params': best_params,
            'cv_score': best_score
        }

# Afficher les résultats de l'optimisation
results_optimized = pd.DataFrame({
    'Modèle': list(optimized_models.keys()),
    'Précision CV optimisée': [model_info['cv_score'] for model_info in optimized_models.values()]
})

display(results_optimized.sort_values('Précision CV optimisée', ascending=False))

## 6. Évaluation des modèles optimisés sur l'ensemble de validation

Évaluons maintenant les performances des modèles optimisés sur l'ensemble de validation.

In [ ]:
# Fonction pour évaluer un modèle sur l'ensemble de validation
def evaluate_on_validation(model, X_train, y_train, X_val, y_val):
    # Entraîner le modèle sur l'ensemble d'entraînement
    model.fit(X_train, y_train)
    
    # Faire des prédictions sur l'ensemble de validation
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculer les métriques
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Calculer l'AUC si possible
    auc = roc_auc_score(y_val, y_pred_proba) if y_pred_proba is not None else None
    
    # Matrice de confusion
    cm = confusion_matrix(y_val, y_pred)
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

# Évaluer chaque modèle optimisé
validation_results = {}

for model_name, model_info in optimized_models.items():
    model = model_info['model']
    results = evaluate_on_validation(model, X_train_split, y_train_split, X_val, y_val)
    validation_results[model_name] = results
    
    print(f"\nRésultats pour {model_name} sur l'ensemble de validation:")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1-score: {results['f1']:.4f}")
    if results['auc'] is not None:
        print(f"  AUC: {results['auc']:.4f}")
    
    print("\n  Matrice de confusion:")
    cm = results['confusion_matrix']
    display(pd.DataFrame(
        cm, 
        index=['Réel: Non-survie (0)', 'Réel: Survie (1)'],
        columns=['Prédit: Non-survie (0)', 'Prédit: Survie (1)']
    ))

# Créer un DataFrame pour comparer les métriques
metrics_df = pd.DataFrame({
    'Modèle': list(validation_results.keys()),
    'Accuracy': [results['accuracy'] for results in validation_results.values()],
    'Precision': [results['precision'] for results in validation_results.values()],
    'Recall': [results['recall'] for results in validation_results.values()],
    'F1-score': [results['f1'] for results in validation_results.values()],
    'AUC': [results['auc'] for results in validation_results.values() if results['auc'] is not None]
})

display(metrics_df.sort_values('Accuracy', ascending=False))

# Visualiser les courbes ROC pour les modèles optimisés
plt.figure(figsize=(10, 8))

for model_name, results in validation_results.items():
    if results['y_pred_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_val, results['y_pred_proba'])
        auc = results['auc']
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--')  # Ligne de référence (aléatoire)
plt.xlabel('Taux de faux positifs', fontsize=12)
plt.ylabel('Taux de vrais positifs', fontsize=12)
plt.title('Courbes ROC des modèles optimisés', fontsize=16)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()

## 7. Sélection du modèle final et entraînement sur toutes les données

Sélectionnons le meilleur modèle et entraînons-le sur l'ensemble complet des données d'entraînement.

In [ ]:
# Sélectionner le meilleur modèle en fonction des performances sur l'ensemble de validation
best_model_name = metrics_df.iloc[0]['Modèle']
best_model = optimized_models[best_model_name]['model']
best_params = optimized_models[best_model_name]['params']

print(f"Le meilleur modèle est: {best_model_name}")
print(f"Paramètres optimaux: {best_params}")

# Entraîner le modèle final sur l'ensemble complet des données d'entraînement
print("\nEntraînement du modèle final sur toutes les données d'entraînement...")
best_model.fit(X_train, y_train)

# Calculer l'importance des caractéristiques si disponible
importance_df = get_feature_importance(best_model, feature_names)

if importance_df is not None:
    print("\nImportance des caractéristiques du modèle final:")
    display(importance_df.head(15))
    
    # Visualiser l'importance des caractéristiques
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=importance_df.head(15), palette='viridis')
    plt.title(f'Importance des caractéristiques du modèle final ({best_model_name})', fontsize=16)
    plt.xlabel('Importance', fontsize=14)
    plt.ylabel('Caractéristique', fontsize=14)
    plt.tight_layout()
    plt.show()

## 8. Générer les prédictions finales pour l'ensemble de test

Utilisons notre modèle final pour faire des prédictions sur l'ensemble de test.

In [ ]:
# Faire des prédictions sur l'ensemble de test
test_predictions = best_model.predict(X_test)
test_probabilities = best_model.predict_proba(X_test)[:, 1] if hasattr(best_model, 'predict_proba') else None

# Créer le DataFrame pour la soumission
submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': test_predictions
})

print("Aperçu des prédictions:")
display(submission.head(10))

# Distribution des prédictions
survived_count = submission['Survived'].sum()
total_count = len(submission)
survival_rate = survived_count / total_count * 100

print(f"\nStatistiques des prédictions:")
print(f"Nombre de passagers prédits comme survivants: {survived_count} sur {total_count} ({survival_rate:.2f}%)")

# Visualiser la distribution des prédictions
plt.figure(figsize=(10, 6))
sns.countplot(x='Survived', data=submission, palette=['#FF5252', '#4CAF50'])
plt.title('Distribution des prédictions de survie', fontsize=16)
plt.xlabel('Prédit comme survivant (0 = Non, 1 = Oui)')
plt.ylabel('Nombre de passagers')

# Ajouter les annotations sur les barres
counts = submission['Survived'].value_counts()
for i, count in enumerate(counts):
    percentage = count / total_count * 100
    plt.text(i, count + 5, f'{count} ({percentage:.1f}%)', ha='center')

plt.tight_layout()
plt.show()

# Sauvegarder le fichier de soumission
submissions_dir = '/workspaces/titanicML/submissions'
if not os.path.exists(submissions_dir):
    os.makedirs(submissions_dir)

submission_file = os.path.join(submissions_dir, f'submission_{best_model_name.replace(" ", "_").lower()}.csv')
submission.to_csv(submission_file, index=False)

print(f"\nFichier de soumission créé: {submission_file}")

## 9. Sauvegarder le modèle final

Sauvegardons le modèle final pour une utilisation ultérieure.

In [ ]:
# Créer un dictionnaire avec le modèle et les métadonnées
final_model_data = {
    'model': best_model,
    'model_name': best_model_name,
    'params': best_params,
    'feature_names': feature_names,
    'scaler': scaler,  # Le scaler utilisé pour normaliser les données
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Créer le dossier pour les modèles s'il n'existe pas
models_dir = '/workspaces/titanicML/models'
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Sauvegarder le modèle
model_file = os.path.join(models_dir, f'final_model_{best_model_name.replace(" ", "_").lower()}.pkl')
joblib.dump(final_model_data, model_file)

print(f"Modèle final sauvegardé: {model_file}")

## 10. Conclusions et prochaines étapes

Dans ce notebook, nous avons:

1. **Testé plusieurs algorithmes de classification**:
   - Régression logistique
   - Arbre de décision
   - Random Forest
   - Gradient Boosting
   - SVM
   - K-Nearest Neighbors
   - XGBoost

2. **Optimisé les hyperparamètres** des modèles les plus prometteurs pour améliorer leurs performances

3. **Évalué les performances des modèles** sur un ensemble de validation à l'aide de plusieurs métriques:
   - Accuracy
   - Precision
   - Recall
   - F1-score
   - AUC-ROC

4. **Sélectionné le meilleur modèle** en fonction de ses performances et l'avons entraîné sur toutes les données d'entraînement

5. **Généré des prédictions** pour l'ensemble de test et créé un fichier de soumission pour Kaggle

### Prochaines étapes:
- **Soumettre nos prédictions à Kaggle** et évaluer notre score
- **Affiner davantage le modèle** en créant de nouvelles caractéristiques ou en essayant des techniques d'ensemble
- **Explorer des approches de stacking** en combinant plusieurs modèles pour améliorer les performances
- **Analyser les erreurs** pour comprendre où notre modèle échoue et comment l'améliorer